In [2]:
import pandas as pd

In [3]:
"""На базе EDA создана функция preprocessing из скрипта preprocessing.py"""

'На базе EDA создана функция preprocessing из скрипта preprocessing.py'

In [4]:
interview_x = pd.read_csv("/Users/victorialokteva/Downloads/interviewdata/interview.X.csv")
interview_y = pd.read_csv("/Users/victorialokteva/Downloads/interviewdata/interview.y.csv")
interview_x.shape, interview_y.shape

((955239, 10), (15359, 2))

In [5]:
interview_x.head()

,reg_time,uid,fc_imp_chk,fc_time_chk,utmtr,mm_dma,osName,model,hardware,site_id
0,2021-07-21 19:25:47,00000000-0000-0001-6268-955448351062,0,7,6,530,iOS,iPhone XR,Mobile Phone,soaps.sheknows.com
1,2021-07-21 19:25:51,00000000-0000-0001-6268-955448381600,0,7,6,612,iOS,iPhone,Mobile Phone,www.forbes.com
2,2021-07-21 19:25:52,00000000-0000-0001-6268-955460621935,0,7,6,524,Android,SM-G998U,Mobile Phone,www.parents.com
3,2021-07-21 19:26:05,00000000-0000-0001-6268-955631601064,0,7,6,659,iOS,iPhone,Mobile Phone,livingforthesunshine.com
4,2021-07-21 19:26:05,00000000-0000-0001-6268-955659731502,0,7,6,505,iOS,iPhone,Mobile Phone,www.pro-football-reference.com


In [6]:
interview_y.head()

,uid,tag
0,00000000-0000-0001-6268-860993461181,fclick
1,00000000-0000-0001-6268-861002231327,fclick
2,00000000-0000-0001-6268-861081671348,fclick
3,00000000-0000-0001-6268-869788991321,fclick
4,00000000-0000-0001-6268-869788451455,fclick


In [7]:
duplicate_count_y = interview_y.duplicated().sum()
duplicate_count_x = interview_x.duplicated().sum()
print(f"interview_x содержит {duplicate_count_x} дупликатов")
print(f"interview_y содержит {duplicate_count_y} дупликатов")


interview_x содержит 24 дупликатов
interview_y содержит 2882 дупликатов


In [8]:

for col in interview_x.columns:
    if interview_x[col].isna().sum() > 0:
        print(f"В колонке {col} {interview_x[col].isna().sum()}  нанов")

В колонке osName 7892  нанов
В колонке model 8395  нанов
В колонке hardware 8185  нанов


In [9]:
# Проверим типы данных. Так как в 3 колонках есть наны, то в них появляется float
for col in interview_x.columns:
    print(col)
    types_in_column = interview_x[col].apply(type)
    print(types_in_column.unique())

    print()

reg_time
[<class 'str'>]

uid
[<class 'str'>]

fc_imp_chk
[<class 'int'>]

fc_time_chk
[<class 'int'>]

utmtr
[<class 'int'>]

mm_dma
[<class 'int'>]

osName
[<class 'str'> <class 'float'>]

model
[<class 'str'> <class 'float'>]

hardware
[<class 'str'> <class 'float'>]

site_id
[<class 'str'>]



In [10]:
# Проверим ли пападают ли данные некоторык колонок в диапазон, указанный в описании данных
print(all(value in range(-1, 5) for value in interview_x['fc_imp_chk'].unique()))
print(all(value in range(-1, 8) for value in interview_x['fc_time_chk'].unique()))
print(all(value in range(-1, 8) for value in interview_x['utmtr'].unique()))

True
True
True


In [11]:
interview_x['mm_dma'].value_counts()
# В колонке mm_dma есть редкие значения -- при препроцессинге объединим их в отдельную категорию -1 (так как все реальные значения неотрицательны)

501    153869
524     96101
517     55164
0       46178
505     43400
        ...  
810         1
771         1
744         1
813         1
636         1
Name: mm_dma, Length: 178, dtype: int64

In [12]:
#Аналогичная ситуация с site_id -- есть сайты, встречающиеся всего 1 раз, тоже будем объединять такие в отдельную категорию
interview_x['site_id'].value_counts().tail()

www.lefigaro.fr        1
rayanworld.com         1
www.routesnorth.com    1
mvlehti.net            1
www.u-tv.ru            1
Name: site_id, dtype: int64

In [13]:
interview_x['hardware'].value_counts()

Mobile Phone                946415
Media Player                   396
Desktop                        208
Refrigerator                    15
Mobile+Phone                     8
Tablet                           6
Data Collection Terminal         5
Digital Home Assistant           1
Name: hardware, dtype: int64

In [14]:
interview_x['site_id'].value_counts()

www.dailymotion.com    150709
whatculture.com        102378
nypost.com              74063
www.digitalspy.com      28385
www.yahoo.com           19036
                        ...  
www.lefigaro.fr             1
rayanworld.com              1
www.routesnorth.com         1
mvlehti.net                 1
www.u-tv.ru                 1
Name: site_id, Length: 11290, dtype: int64

In [172]:
interview_x['site_id'] = interview_x['site_id'].map(lambda x: x.replace("www1.", "").replace("www.", "").replace("co.uk", "co_uk"))

interview_x['len'] = interview_x['site_id'].map(lambda x: len(x.split(".")))
interview_x['first_word'] = interview_x['site_id'].map(lambda x: x.split(".")[0])

interview_x['sport_cat'] = (interview_x['site_id'].str.contains('sport')
                            | interview_x['site_id'].str.contains('baseball')
                            | interview_x['site_id'].str.contains('football')
                            | interview_x['site_id'].str.contains('soccer')
                            | interview_x['site_id'].str.contains('basketball')
                            | interview_x['site_id'].str.contains('hockey')
                            | interview_x['site_id'].str.contains('moneyball')
                            | interview_x['site_id'].str.contains('rotowire')
                            | interview_x['site_id'].str.contains('mmafighting')
                            | interview_x['site_id'].str.contains('mmamania')
                            | interview_x['site_id'].str.contains('nbaanalysis')
                            | interview_x['site_id'].str.contains('nba-trade-rumors')
                            | interview_x['site_id'].str.contains('nfltraderumors')
                            | interview_x['site_id'].str.contains('nfldraftdiamonds')
                            | interview_x['site_id'].str.contains('detroitbadboys.com') #?
                            | interview_x['site_id'].str.contains('volleyball'))

interview_x['travel_cat'] = (interview_x['site_id'].str.contains('travel')
                             | interview_x['site_id'].str.contains('tourist'))
interview_x['weather_cat'] = (interview_x['site_id'].str.contains('weather'))
interview_x['game_cat'] = (interview_x['site_id'].str.contains('game'))
interview_x['music_cat'] = (interview_x['site_id'].str.contains('music')
                            | interview_x['site_id'].str.contains('hip-hop')
                            | interview_x['site_id'].str.contains('song') # ?
                            | interview_x['site_id'].str.contains('guitar'))

interview_x['cook_cat'] = (interview_x['site_id'].str.contains('cook')
                           | interview_x['site_id'].str.contains('diet')
                           | interview_x['site_id'].str.contains('fastfood')
                           | interview_x['site_id'].str.contains('cupcake')  # javacupcake.com
                           | interview_x['site_id'].str.contains('grandbaby-cakes')
                           | interview_x['site_id'].str.contains('browneyedbaker')  # baker?
                           | interview_x['site_id'].str.contains('baking')
                           | interview_x['site_id'].str.contains('lovebakesgoodcakes')
                           | interview_x['site_id'].str.contains('bakery'))

interview_x['health_cat'] = (interview_x['site_id'].str.contains('health')
                             | interview_x['site_id'].str.contains('medical'))

interview_x['zodiac_cat'] = (interview_x['site_id'].str.contains('astrology')
                             | interview_x['site_id'].str.contains('zodiac'))

interview_x['no_cat'] = ((~interview_x['sport_cat'])
                         & (~interview_x['travel_cat'])
                         & (~interview_x['health_cat'])
                         & (~interview_x['game_cat'])
                         & (~interview_x['music_cat'])
                         & (~interview_x['cook_cat'])
                         & (~interview_x['zodiac_cat'])
                         & (~interview_x['weather_cat']))

In [173]:
interview_x.loc[interview_x['no_cat'], 'site_id'].shape

(902941,)

In [174]:
interview_x.loc[interview_x['no_cat'], 'site_id'].value_counts()

dailymotion.com            150709
whatculture.com            102378
nypost.com                  74063
digitalspy.com              28385
yahoo.com                   19169
                            ...  
lovetextmessages.com.ng         1
bronx.news12.com                1
london.eater.com                1
loveleafco.com                  1
u-tv.ru                         1
Name: site_id, Length: 10272, dtype: int64

In [175]:
interview_x.loc[interview_x['len'] > 2, 'first_word'].value_counts()

my                        15926
finance                   12534
news                       7427
currently                  7228
metropcs                   7136
                          ...  
bbs                           1
overlordmaruyama              1
brasil                        1
media                         1
estrenos-series-espana        1
Name: first_word, Length: 1043, dtype: int64

In [177]:
interview_x.loc[interview_x['no_cat']
                & interview_x['site_id'].str.contains('recipe')]['site_id'].value_counts()

insanelygoodrecipes.com         1292
allrecipes.com                   796
therecipecritic.com              244
letsdishrecipes.com              233
thereciperebel.com               200
                                ... 
robustrecipes.com                  1
myrecipetreasures.com              1
recipeswithessentialoils.com       1
norecipes.com                      1
therecipewell.com                  1
Name: site_id, Length: 108, dtype: int64

In [190]:
interview_x['reg_time'].map(lambda x: x.year)

AttributeError: 'str' object has no attribute 'year'

In [186]:
interview_x['domain'] = interview_x['site_id'].map(lambda x:  x.split('.')[-1])

0         com
1         com
2         com
3         com
4         com
         ... 
955234    com
955235    com
955236    com
955237    com
955238    com
Name: site_id, Length: 955239, dtype: object

In [182]:
interview_x['site_id'].str.split('.')

0                [soaps, sheknows, com]
1                         [forbes, com]
2                        [parents, com]
3           [livingforthesunshine, com]
4         [pro-football-reference, com]
                      ...              
955234               [forsythnews, com]
955235     [currently, att, yahoo, com]
955236                  [ladbible, com]
955237                  [ladbible, com]
955238                  [ladbible, com]
Name: site_id, Length: 955239, dtype: object

In [ ]:
# amp.